# Level 5: Advanced SQL Queries

This notebook moves beyond simple data retrieval to cover more advanced querying techniques. We'll explore how to aggregate data, group it, combine data from multiple tables using joins, and use subqueries and views to structure complex logic.

### Setup
For this notebook, we need a more complex schema with multiple related tables. We'll create an `authors` table and a `books` table with a foreign key relationship.

In [1]:
import sqlite3
import os

db_file = 'advanced_queries.db'
if os.path.exists(db_file):
    os.remove(db_file)

conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Create authors table
cursor.execute("""
CREATE TABLE authors (
    author_id INTEGER PRIMARY KEY,
    author_name TEXT NOT NULL
);
""")

# Create books table with a foreign key to authors
cursor.execute("""
CREATE TABLE books (
    book_id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    publish_year INTEGER,
    author_id INTEGER,
    FOREIGN KEY (author_id) REFERENCES authors (author_id)
);
""")

# Populate tables
authors_data = [(1, 'J.R.R. Tolkien'), (2, 'Frank Herbert'), (3, 'George Orwell')]
books_data = [
    ('The Hobbit', 1937, 1),
    ('The Lord of the Rings', 1954, 1),
    ('Dune', 1965, 2),
    ('1984', 1949, 3),
    ('Animal Farm', 1945, 3)
]

cursor.executemany("INSERT INTO authors VALUES (?, ?)", authors_data)
cursor.executemany("INSERT INTO books (title, publish_year, author_id) VALUES (?, ?, ?)", books_data)
conn.commit()

## 5.1 Aggregation Functions

Aggregation functions perform a calculation on a set of values and return a single value.

In [2]:
cursor.execute("SELECT COUNT(*) FROM books;")
print(f"Total number of books: {cursor.fetchone()[0]}")

cursor.execute("SELECT AVG(publish_year) FROM books;")
print(f"Average publish year: {cursor.fetchone()[0]:.0f}")

cursor.execute("SELECT MIN(publish_year) FROM books;")
print(f"Earliest publish year: {cursor.fetchone()[0]}")

Total number of books: 5
Average publish year: 1950
Earliest publish year: 1937


## 5.2 Grouping Data (`GROUP BY` and `HAVING`)

The `GROUP BY` statement groups rows that have the same values in specified columns into summary rows. It's often used with aggregate functions.

The `HAVING` clause is used to filter these groups, whereas `WHERE` filters individual rows *before* grouping.

In [3]:
# Count the number of books per author
query = """
SELECT author_id, COUNT(book_id) as num_books
FROM books
GROUP BY author_id;
"""
cursor.execute(query)
print("Number of books per author:", cursor.fetchall())

Number of books per author: [(1, 2), (2, 1), (3, 2)]


In [4]:
# Find authors who have written more than one book
query = """
SELECT author_id, COUNT(book_id) as num_books
FROM books
GROUP BY author_id
HAVING COUNT(book_id) > 1;
"""
cursor.execute(query)
print("\nAuthors with more than one book:", cursor.fetchall())


Authors with more than one book: [(1, 2), (3, 2)]


## 5.3 Joins

`JOIN` clauses are used to combine rows from two or more tables based on a related column between them.

### `INNER JOIN`
Returns records that have matching values in both tables.

In [5]:
query = """
SELECT books.title, authors.author_name
FROM books
INNER JOIN authors ON books.author_id = authors.author_id;
"""
cursor.execute(query)
print("Books with their authors:", cursor.fetchall())

Books with their authors: [('The Hobbit', 'J.R.R. Tolkien'), ('The Lord of the Rings', 'J.R.R. Tolkien'), ('Dune', 'Frank Herbert'), ('1984', 'George Orwell'), ('Animal Farm', 'George Orwell')]


### `LEFT JOIN`
Returns all records from the left table, and the matched records from the right table. The result is `NULL` from the right side if there is no match.

In [6]:
# Let's add an author who has no books yet
cursor.execute("INSERT INTO authors VALUES (?, ?)", (4, 'Jane Austen'))
conn.commit()

query = """
SELECT authors.author_name, books.title
FROM authors
LEFT JOIN books ON authors.author_id = books.author_id;
"""
cursor.execute(query)
print("All authors and their books (if any):", cursor.fetchall())

All authors and their books (if any): [('J.R.R. Tolkien', 'The Hobbit'), ('J.R.R. Tolkien', 'The Lord of the Rings'), ('Frank Herbert', 'Dune'), ('George Orwell', '1984'), ('George Orwell', 'Animal Farm'), ('Jane Austen', None)]


## 5.4 Subqueries

A subquery is a query nested inside another query. It can be used in `WHERE`, `FROM`, or `SELECT` clauses.

In [7]:
# Find all books written by 'George Orwell'
query = """
SELECT title FROM books
WHERE author_id = (SELECT author_id FROM authors WHERE author_name = 'George Orwell');
"""
cursor.execute(query)
print("Books by George Orwell:", cursor.fetchall())

Books by George Orwell: [('1984',), ('Animal Farm',)]


## 5.5 Views

A view is a virtual table based on the result-set of an SQL statement. It's a stored query that you can interact with as if it were a table. This is useful for simplifying complex queries.

In [8]:
# Create a view that combines book and author information
cursor.execute("""
CREATE VIEW book_details AS
SELECT b.title, b.publish_year, a.author_name
FROM books b
JOIN authors a ON b.author_id = a.author_id;
""")
print("View 'book_details' created.")

View 'book_details' created.


In [9]:
# Now we can query the view like a regular table
cursor.execute("SELECT * FROM book_details WHERE publish_year < 1950;")
print("\nQuerying the view:", cursor.fetchall())


Querying the view: [('The Hobbit', 1937, 'J.R.R. Tolkien'), ('1984', 1949, 'George Orwell'), ('Animal Farm', 1945, 'George Orwell')]


In [10]:
conn.close()